In [1]:
import pymongo
from pymongo import MongoClient

from pandas import DataFrame, read_csv
import json
import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib #only needed to determine Matplotlib version number

client = MongoClient('mongodb://127.0.0.1:27017')
db = client.sample



In [36]:
db.drop_collection('test')
db.test.insert_one({'username':'yyy3yy'})
db.test.insert_one({'username':'arestov_design'})
db.test.insert_one({'username':'space307'})
db.test.insert_one({'username':'Hido'})

In [16]:
cursor = db.test.find()
for elem in cursor:
    print elem

{u'_id': ObjectId('5b65b50ae2850512525a4a74'), u'id': 7, u'nome': u'Canyon 123'}


In [11]:
myquery = { "id": 7 }
newvalues = { "$set": { "nome": "pino", "id": 3 } }
db.test.update_one(myquery, newvalues)



In [13]:
db.test.remove( { "nome": "pino" } )

/home/simo/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  """Entry point for launching an IPython kernel.


{u'n': 1, u'ok': 1.0}

# FOLLOWERS , sostituisco username con id su db

In [11]:
Location = r'/home/simo/UnInfo/Tesi Magistrale/Dataset/elaborati/followers_df.csv'
df = pd.read_csv(Location)

desc_followers = df[df.columns[2]]
desc_user = df[df.columns[1]]

translation_map = db.user_translate.find()

In [12]:
def transalte(username, translation_map):
    for user in translation_map:
        if user['username'] == username:
            return user['id']
    return -1

In [14]:
%%time
print transalte('AlessandraRecine', translation_map)

77820
CPU times: user 768 ms, sys: 40 ms, total: 808 ms
Wall time: 880 ms


In [42]:
# SOLUZIONE CHE MODIFICA I VALORI GIA CARICATI SU DB

# cerco nella mappa di traduzione 'user_translate' se la corrispondenza username-id esiste.
# se esiste sostituisco, altrimenti elimino il record da followers
def translateUsernames(collection):
    try:
        cursorF = collection.find()
        for elem in cursorF:
            user = db.user_translate.find( { 'username': elem['username'] } )
            if user.count()>0 :
                myquery = { "username": elem['username'] }
                newvalues = { "$set": { "username": user[0]['id'] } }
                collection.update_one(myquery, newvalues)
            else:
                collection.remove( { "username": elem['username'] } )
    except Exception as e:
        print e
        print elem

In [8]:


translateUsername(db.followers)
print db.followers.find().count()


NameError: name 'translateUsername' is not defined

In [69]:
print db.followers.find()[3200]

{u'username': 1757871, u'_id': ObjectId('5b65ca42e285051e11c0e217'), u'followed': []}


# affidabilità del 'likes_count' di uno shot

In [20]:
db.shots.find()[771256]

{u'_id': ObjectId('5b62dd4ce285051834e1a4cc'),
 u'shots': [{u'animated': False,
   u'comments_count': 2,
   u'created_at': 1514882568,
   u'description': u"<p>Hello, There.</p>\n\n<p>Recently I've been working on iMessage Sticker app.\n<br />And these are results.</p>\n\n<p>Thanks for any comments. Happy New Year!</p>",
   u'height': 300,
   u'id': 4058523,
   u'likes_count': 43,
   u'tags': [u'emoji', u'illustration', u'imessage sticker'],
   u'team': None,
   u'title': u'Cheezmoji - iMessage Sticker',
   u'updated_at': 1514882829,
   u'views_count': 216,
   u'width': 400}],
 u'username': u'fatty_thumb'}

In [35]:
%%time

cursor_like = db.likes.find({'post': 4058786})
print len(cursor_like[0]['likes'])



46
CPU times: user 64 ms, sys: 8 ms, total: 72 ms
Wall time: 1min 35s


In [44]:
%%time

for i in range(771254, 771258):
    for shot in db.shots.find()[i]['shots']:
        print 'ID: ', shot['id'], ' likes_conut: ', shot['likes_count']
        cursor_like = db.likes.find({'post': shot['id']})
        print "ID post: ", cursor_like[0]['post'], "likes: ", len(cursor_like[0]['likes'])
        print '\n'


 ID:  4057420  likes_conut:  44
ID post:  4057420 likes:  44


ID:  4031313  likes_conut:  18
ID post:  4031313 likes:  18


ID:  4028871  likes_conut:  27
ID post:  4028871 likes:  22


ID:  4012006  likes_conut:  25
ID post:  4012006 likes:  25


ID:  3778521  likes_conut:  26
ID post:  3778521 likes:  26


ID:  3770913  likes_conut:  33
ID post:  3770913 likes:  31


ID:  3763583  likes_conut:  19
ID post:  3763583 likes:  21


ID:  3688512  likes_conut:  114
ID post:  3688512 likes:  113


ID:  4058523  likes_conut:  43
ID post:  4058523 likes:  43


ID:  4058990  likes_conut:  21
ID post:  4058990 likes:  21


ID:  4056657  likes_conut:  12
ID post:  4056657 likes:  12


ID:  4055374  likes_conut:  14
ID post:  4055374 likes:  14


ID:  4053795  likes_conut:  13
ID post:  4053795 likes:  13


ID:  4049411  likes_conut:  12
ID post:  4049411 likes:  12


ID:  4045712  likes_conut:  15
ID post:  4045712 likes:  15


ID:  4043786  likes_conut:  13
ID post:  4043786 likes:  13


ID:  

KeyboardInterrupt: 

# affidabilità dei 'count' di uno user

In [12]:
cursor_user = db.users.find()

In [10]:
cursor_user[0]

{u'_id': ObjectId('5b5b4e81e285053f142e3daf'),
 u'bio': u'Web &amp; UX/UI designer arestov.design@gmail.com',
 u'buckets_count': 5,
 u'comments_received_count': 101,
 u'created_at': 1416216907,
 u'followers_count': 1106,
 u'followings_count': 746,
 u'id': 695302,
 u'likes_count': 5456,
 u'likes_received_count': 4872,
 u'location': u'Moscow, Russia',
 u'name': u'Leonid Arestov',
 u'projects_count': 4,
 u'rebounds_received_count': 7,
 u'shots_count': 65,
 u'teams_count': 0,
 u'updated_at': 1513060644,
 u'username': u'arestov_design'}

### followers_count

In [13]:
print 'user: ', cursor_user[0]['username'], ' followers: ', cursor_user[0]['followers_count'], ' followers count: ', len(cursor_followers[0]['followers'])

user:  arestov_design  followers:  1106  followers count: 

NameError: name 'cursor_followers' is not defined

In [6]:
diff = 0
a = 1900
b = 2400
for i in range(a, b):
    cursor_followers = db.followers.find({'username': cursor_user[i]['username']})
    print 'user: ', cursor_user[i]['username'], '\t\t followers: ', cursor_user[i]['followers_count'], '\t\t followers count: ', len(cursor_followers[0]['followers'])

    


 user:  zhoupeipei 		 followers:  0 		 followers count:  0
user:  Stillioo 		 followers:  0 		 followers count:  0
user:  sophpoz 		 followers:  61 		 followers count:  62
user:  davidleesid 		 followers:  1 		 followers count:  1
user:  hrm 		 followers:  380 		 followers count:  382
user:  gnzndrs 		 followers:  0 		 followers count:  0
user:  Kulaev 		 followers:  0 		 followers count:  0
user:  KissyK 		 followers:  0 		 followers count:  0
user:  wa5killa 		 followers:  0 		 followers count:  0
user:  dokarpov 		 followers:  201 		 followers count:  201
user:  evasiskovaa 		 followers:  31 		 followers count:  31
user:  srinivasan-rajan 		 followers:  841 		 followers count:  839
user:  aias 		 followers:  0 		 followers count:  0
user:  Trajkoski 		 followers:  44 		 followers count:  45
user:  Farbanti 		 followers:  547 		 followers count:  546
user:  chenvoice 		 followers:  0 		 followers count:  0
user:  Lilou11 		 followers:  0 		 followers count:  0
user:  Karachevskiy 		 

user:  imyvonne3 		 followers:  328 		 followers count:  327
user:  ArcticSquire 		 followers:  121 		 followers count:  122
user:  KyleCraven94 		 followers:  463 		 followers count:  464
user:  mikeaiello 		 followers:  101 		 followers count:  101
user:  corneliousx 		 followers:  0 		 followers count:  0
user:  sachivpvlns 		 followers:  0 		 followers count:  0
user:  raghda 		 followers:  0 		 followers count:  0
user:  raphaeldutoit 		 followers:  79 		 followers count:  79
user:  Yueyuefeng 		 followers:  0 		 followers count:  0
user:  kuhn 		 followers:  0 		 followers count:  0
user:  akbarwibawa 		 followers:  0 		 followers count:  0
user:  ismaelbranco 		 followers:  246 		 followers count:  247
user:  kennymax 		 followers:  1084 		 followers count:  1086
user:  jeremylemardele 		 followers:  0 		 followers count:  0
user:  yunique720 		 followers:  7 		 followers count:  7
user:  SeanXing 		 followers:  0 		 followers count:  0
user:  luciarodriguezs 		 followers:  2 		

user:  christinesoules 		 followers:  486 		 followers count:  488
user:  Erika_93 		 followers:  0 		 followers count:  0
user:  nanonansen 		 followers:  0 		 followers count:  0
user:  danyvong 		 followers:  0 		 followers count:  0
user:  CZLpicasso 		 followers:  1 		 followers count:  1
user:  martinvalenta 		 followers:  0 		 followers count:  0
user:  mihovil 		 followers:  74 		 followers count:  74
user:  musterionarmy 		 followers:  34 		 followers count:  34
user:  396147106lilei 		 followers:  0 		 followers count:  0
user:  bhdigital 		 followers:  0 		 followers count:  0
user:  Kisoo 		 followers:  0 		 followers count:  0
user:  cTermit 		 followers:  151 		 followers count:  151
user:  srinivasdhotre16 		 followers:  473 		 followers count:  475
user:  robinxing 		 followers:  0 		 followers count:  0
user:  afriis 		 followers:  0 		 followers count:  0
user:  jessegresham 		 followers:  0 		 followers count:  0
user:  Baranchikov 		 followers:  10 		 followers coun

user:  rifsroo 		 followers:  180 		 followers count:  181
user:  Designoholic 		 followers:  341 		 followers count:  341
user:  henrydoe 		 followers:  3034 		 followers count:  3037
user:  Atheek 		 followers:  134 		 followers count:  234
user:  boutchen 		 followers:  0 		 followers count:  0
user:  aliceholm89 		 followers:  0 		 followers count:  0
user:  pablomajernik 		 followers:  18 		 followers count:  18
user:  madebyjaz 		 followers:  16 		 followers count:  16
user:  denysergushkin 		 followers:  452 		 followers count:  553
user:  russell-hepton 		 followers:  42 		 followers count:  42
user:  satya_rajpurohit 		 followers:  113 		 followers count:  113
user:  lightscientist 		 followers:  67 		 followers count:  67
user:  telldaren 		 followers:  0 		 followers count:  0
user:  lauracoggins 		 followers:  1075 		 followers count:  0
user:  lukeriksen 		 followers:  0 		 followers count:  0
user:  jessedymond 		 followers:  0 		 followers count:  0
user:  samil 		 follo

### likes_received_count

In [11]:
def printShotsLike(username):
    shots = db.shots.find({'username': username})

    num_likes = 0
    for s in shots[0]['shots']:
        num_likes = num_likes + s['likes_count']

    print 'shots likes: ', num_likes, '\n'

In [12]:
user = db.users.find()
for i in range(100,200):
    print 'user: ', user[i]['username'], ' - like received: ', user[i]['likes_received_count']
    printShotsLike(user[i]['username'])

user:  visp  - like received:  3221
shots likes:  3222 

user:  chevail  - like received:  0
shots likes:  0 

user:  Sohieb  - like received:  0
shots likes:  0 

user:  minqi_luo  - like received:  296
shots likes:  296 

user:  owerrc  - like received:  0
shots likes:  0 

user:  YanboSolutions  - like received:  0
shots likes:  0 

user:  ricardogiesta  - like received:  0
shots likes:  0 

user:  santiagovaq  - like received:  307
shots likes:  306 

user:  JiarenHui  - like received:  172
shots likes:  173 

user:  liaoevan  - like received:  376
shots likes:  375 

user:  Viknayak  - like received:  0
shots likes:  0 

user:  XD  - like received:  642
shots likes:  642 

user:  OmarMamdouh25  - like received:  0
shots likes:  0 

user:  adamkporter  - like received:  1086
shots likes:  1087 

user:  lucschwab  - like received:  709
shots likes:  709 

user:  noelanijing  - like received:  1746
shots likes:  1747 

user:  shadsahm  - like received:  649
shots likes:  649 

user: 

### shots_count

In [14]:
def printShotsCount(username):    
    shots = db.shots.find({'username': username})
    print 'shots count: ', len(shots[0]['shots']), '\n'

In [15]:
user = db.users.find()
for i in range(100,200):
    print 'user: ', user[i]['username'], ' - user shots: ', user[i]['shots_count']
    printShotsCount(user[i]['username'])

user:  visp  - user shots:  21
shots count:  21 

user:  chevail  - user shots:  0
shots count:  0 

user:  Sohieb  - user shots:  0
shots count:  0 

user:  minqi_luo  - user shots:  16
shots count:  16 

user:  owerrc  - user shots:  0
shots count:  0 

user:  YanboSolutions  - user shots:  0
shots count:  0 

user:  ricardogiesta  - user shots:  0
shots count:  0 

user:  santiagovaq  - user shots:  23
shots count:  23 

user:  JiarenHui  - user shots:  12
shots count:  12 

user:  liaoevan  - user shots:  27
shots count:  27 

user:  Viknayak  - user shots:  0
shots count:  0 

user:  XD  - user shots:  28
shots count:  28 

user:  OmarMamdouh25  - user shots:  0
shots count:  0 

user:  adamkporter  - user shots:  39
shots count:  39 

user:  lucschwab  - user shots:  58
shots count:  58 

user:  noelanijing  - user shots:  39
shots count:  39 

user:  shadsahm  - user shots:  25
shots count:  25 

user:  mgsong  - user shots:  0
shots count:  0 

user:  felipemendes  - user shots

### likes_count

In [24]:
%%time

cursor_likes = db.likes.find()

like_count = 0

for elem in cursor_likes:
    for like in elem['likes']:
        if like['username'] == 'shivansh':
            like_count = like_count+1
            
print like_count

KeyboardInterrupt: 

### followings_count

In [36]:
def printFollowingsCount(username):
    cursor_followers = db.followers.find()

    foll_count = 0

    for elem in cursor_followers:
        for foll in elem['followers']:
            if foll['follower_user'] == username:
                foll_count = foll_count+1

    print 'count in followers dataset:', foll_count, '\n'

In [37]:
%%time
user = db.users.find()
for i in range(100,200):
    print 'user: ', user[i]['username'], ' - user followings: ', user[i]['followings_count']
    printFollowingsCount(user[i]['username'])

user:  visp  - user followings:  1023
count in followers dataset: 1062 

user:  chevail  - user followings:  72
count in followers dataset: 66 

user:  Sohieb  - user followings:  93
count in followers dataset: 92 

user:  minqi_luo  - user followings:  93
count in followers dataset: 90 

user:  owerrc  - user followings:  84
count in followers dataset: 79 

user:  YanboSolutions  - user followings:  33
count in followers dataset: 32 

user:  ricardogiesta  - user followings:  1070
count in followers dataset: 1062 

user:  santiagovaq  - user followings:  494
count in followers dataset: 488 

user:  JiarenHui  - user followings:  69
count in followers dataset: 69 

user:  liaoevan  - user followings:  328
count in followers dataset: 314 

user:  Viknayak  - user followings:  1
count in followers dataset: 0 

user:  XD  - user followings:  779
count in followers dataset: 757 

user:  OmarMamdouh25  - user followings:  207
count in followers dataset: 209 

user:  adamkporter  - user foll

# Omologazione dei dati sulla data di crawling

### user dataset - followers_count

In [44]:
cursor_user = db.users.find({'created_at': {'$lt':1512950340}})


In [45]:
#cursor_user = db.users.find({'created_at': {'$lt':1515369540}})
#print cursor_user.count()

In [46]:
diff = 0
a = 1900
b = 2000

count = 0

for i in range(a, b):
    cursor_followers = db.followers.find({'username': cursor_user[i]['username']})
    for elem in cursor_followers:
        for f in elem['followers']:
            if f['timestamp'] <= 1512950340:
                count = count+1
        
    print 'user: ', cursor_user[i]['username'], '\t\t followers: ', cursor_user[i]['followers_count'], '\t\t followers count: ', count
    count = 0
    

user:  zhoupeipei 		 followers:  0 		 followers count:  0
user:  Stillioo 		 followers:  0 		 followers count:  0
user:  sophpoz 		 followers:  61 		 followers count:  58
user:  davidleesid 		 followers:  1 		 followers count:  1
user:  hrm 		 followers:  380 		 followers count:  380
user:  gnzndrs 		 followers:  0 		 followers count:  0
user:  Kulaev 		 followers:  0 		 followers count:  0
user:  KissyK 		 followers:  0 		 followers count:  0
user:  wa5killa 		 followers:  0 		 followers count:  0
user:  dokarpov 		 followers:  201 		 followers count:  201
user:  evasiskovaa 		 followers:  31 		 followers count:  31
user:  srinivasan-rajan 		 followers:  841 		 followers count:  838
user:  aias 		 followers:  0 		 followers count:  0
user:  Trajkoski 		 followers:  44 		 followers count:  44
user:  Farbanti 		 followers:  547 		 followers count:  546
user:  chenvoice 		 followers:  0 		 followers count:  0
user:  Lilou11 		 followers:  0 		 followers count:  0
user:  Karachevskiy 		 f

### user dataset - likes_count

In [35]:
print db.users.find({'username': 'space307'})[0]['likes_count']

2447


In [57]:
%%time

#cursor_user = db.users.find({'created_at': {'$lt':1513209540}, 'username': 'space307'})
#print cursor_user.count()

cursor_likes = db.likes.find()

like_count = 0

for elem in cursor_likes:
    for like in elem['likes']:
        if like['username'] == 'space307' and like['timestamp'] <= 1513209540:
            like_count = like_count+1
            
print like_count

{u'username': u'space307', u'timestamp': 1511511046, u'like_id': 102624558}
{u'username': u'space307', u'timestamp': 1512481280, u'like_id': 103727389}
{u'username': u'space307', u'timestamp': 1513063741, u'like_id': 104399385}
{u'username': u'space307', u'timestamp': 1511258514, u'like_id': 102314102}
{u'username': u'space307', u'timestamp': 1510834714, u'like_id': 101873490}
{u'username': u'space307', u'timestamp': 1511863507, u'like_id': 102965857}
{u'username': u'space307', u'timestamp': 1510839181, u'like_id': 101881483}
{u'username': u'space307', u'timestamp': 1510832114, u'like_id': 101870210}
{u'username': u'space307', u'timestamp': 1512565332, u'like_id': 103849361}
{u'username': u'space307', u'timestamp': 1512481298, u'like_id': 103727434}
{u'username': u'space307', u'timestamp': 1510834723, u'like_id': 101873500}
{u'username': u'space307', u'timestamp': 1510315773, u'like_id': 101312373}
{u'username': u'space307', u'timestamp': 1510832112, u'like_id': 101870205}
{u'username'

{u'username': u'space307', u'timestamp': 1510687092, u'like_id': 101683699}
{u'username': u'space307', u'timestamp': 1510951321, u'like_id': 102026873}
{u'username': u'space307', u'timestamp': 1511284346, u'like_id': 102361157}
{u'username': u'space307', u'timestamp': 1511284348, u'like_id': 102361160}
{u'username': u'space307', u'timestamp': 1511284353, u'like_id': 102361170}
{u'username': u'space307', u'timestamp': 1511284354, u'like_id': 102361173}
{u'username': u'space307', u'timestamp': 1511511078, u'like_id': 102624650}
{u'username': u'space307', u'timestamp': 1510919306, u'like_id': 101983104}
{u'username': u'space307', u'timestamp': 1513095053, u'like_id': 104458248}
{u'username': u'space307', u'timestamp': 1512565337, u'like_id': 103849377}
{u'username': u'space307', u'timestamp': 1512475503, u'like_id': 103715663}
{u'username': u'space307', u'timestamp': 1513095067, u'like_id': 104458340}
{u'username': u'space307', u'timestamp': 1511258438, u'like_id': 102313863}
{u'username'

{u'username': u'space307', u'timestamp': 1511261323, u'like_id': 102320367}
{u'username': u'space307', u'timestamp': 1511261116, u'like_id': 102319903}
{u'username': u'space307', u'timestamp': 1512565335, u'like_id': 103849372}
{u'username': u'space307', u'timestamp': 1511442202, u'like_id': 102557259}
{u'username': u'space307', u'timestamp': 1510214177, u'like_id': 101170650}
{u'username': u'space307', u'timestamp': 1510919295, u'like_id': 101983084}
{u'username': u'space307', u'timestamp': 1511440865, u'like_id': 102554901}
{u'username': u'space307', u'timestamp': 1512481573, u'like_id': 103728026}
{u'username': u'space307', u'timestamp': 1510512475, u'like_id': 101447668}
{u'username': u'space307', u'timestamp': 1511429417, u'like_id': 102534981}
{u'username': u'space307', u'timestamp': 1511463141, u'like_id': 102586242}
{u'username': u'space307', u'timestamp': 1511463146, u'like_id': 102586245}
{u'username': u'space307', u'timestamp': 1511435097, u'like_id': 102545092}
{u'username'

{u'username': u'space307', u'timestamp': 1512502804, u'like_id': 103766047}
{u'username': u'space307', u'timestamp': 1512564855, u'like_id': 103848412}
{u'username': u'space307', u'timestamp': 1512481284, u'like_id': 103727402}
{u'username': u'space307', u'timestamp': 1511863500, u'like_id': 102965834}
{u'username': u'space307', u'timestamp': 1511438120, u'like_id': 102550183}
{u'username': u'space307', u'timestamp': 1511511095, u'like_id': 102624688}
{u'username': u'space307', u'timestamp': 1510819659, u'like_id': 101844932}
{u'username': u'space307', u'timestamp': 1513063675, u'like_id': 104399265}
{u'username': u'space307', u'timestamp': 1510839200, u'like_id': 101881536}
{u'username': u'space307', u'timestamp': 1513063754, u'like_id': 104399433}
{u'username': u'space307', u'timestamp': 1511522424, u'like_id': 102643694}
{u'username': u'space307', u'timestamp': 1511258446, u'like_id': 102313885}
{u'username': u'space307', u'timestamp': 1510819012, u'like_id': 101843150}
{u'username'

{u'username': u'space307', u'timestamp': 1512565343, u'like_id': 103849388}
{u'username': u'space307', u'timestamp': 1510818963, u'like_id': 101842938}
{u'username': u'space307', u'timestamp': 1510819209, u'like_id': 101843714}
{u'username': u'space307', u'timestamp': 1510687133, u'like_id': 101683775}
{u'username': u'space307', u'timestamp': 1510919253, u'like_id': 101983020}
{u'username': u'space307', u'timestamp': 1511511089, u'like_id': 102624670}
{u'username': u'space307', u'timestamp': 1512847536, u'like_id': 104183735}
{u'username': u'space307', u'timestamp': 1510687150, u'like_id': 101683803}
{u'username': u'space307', u'timestamp': 1511522393, u'like_id': 102643626}
{u'username': u'space307', u'timestamp': 1510912002, u'like_id': 101971648}
{u'username': u'space307', u'timestamp': 1512475496, u'like_id': 103715647}
{u'username': u'space307', u'timestamp': 1511442445, u'like_id': 102557722}
{u'username': u'space307', u'timestamp': 1512475568, u'like_id': 103715808}
{u'username'

...

{u'username': u'space307', u'timestamp': 1511343416, u'like_id': 102427699}
{u'username': u'space307', u'timestamp': 1509693009, u'like_id': 100612148}
{u'username': u'space307', u'timestamp': 1510373854, u'like_id': 101369838}
{u'username': u'space307', u'timestamp': 1510909526, u'like_id': 101967450}
{u'username': u'space307', u'timestamp': 1510235036, u'like_id': 101208289}
{u'username': u'space307', u'timestamp': 1510919290, u'like_id': 101983072}
{u'username': u'space307', u'timestamp': 1511284330, u'like_id': 102361126}
{u'username': u'space307', u'timestamp': 1510512707, u'like_id': 101447840}
{u'username': u'space307', u'timestamp': 1512732848, u'like_id': 104082303}
{u'username': u'space307', u'timestamp': 1511432750, u'like_id': 102541140}
{u'username': u'space307', u'timestamp': 1510817444, u'like_id': 101840319}
{u'username': u'space307', u'timestamp': 1511858398, u'like_id': 102955686}
{u'username': u'space307', u'timestamp': 1511511097, u'like_id': 102624689}
{u'username'

{u'username': u'space307', u'timestamp': 1511261521, u'like_id': 102320773}
{u'username': u'space307', u'timestamp': 1512475525, u'like_id': 103715718}
{u'username': u'space307', u'timestamp': 1513095038, u'like_id': 104458213}
{u'username': u'space307', u'timestamp': 1512724686, u'like_id': 104069164}
{u'username': u'space307', u'timestamp': 1513095043, u'like_id': 104458219}
{u'username': u'space307', u'timestamp': 1511210927, u'like_id': 102254505}
{u'username': u'space307', u'timestamp': 1511858450, u'like_id': 102955796}
{u'username': u'space307', u'timestamp': 1510687097, u'like_id': 101683709}
{u'username': u'space307', u'timestamp': 1511118302, u'like_id': 102127459}
{u'username': u'space307', u'timestamp': 1510373810, u'like_id': 101369818}
{u'username': u'space307', u'timestamp': 1512565319, u'like_id': 103849326}
{u'username': u'space307', u'timestamp': 1510839188, u'like_id': 101881500}
{u'username': u'space307', u'timestamp': 1510832104, u'like_id': 101870184}
{u'username'

{u'username': u'space307', u'timestamp': 1511441764, u'like_id': 102556491}
{u'username': u'space307', u'timestamp': 1510310577, u'like_id': 101304520}
{u'username': u'space307', u'timestamp': 1511438075, u'like_id': 102550087}
{u'username': u'space307', u'timestamp': 1510140054, u'like_id': 101075323}
{u'username': u'space307', u'timestamp': 1511511076, u'like_id': 102624645}
{u'username': u'space307', u'timestamp': 1510830500, u'like_id': 101867573}
{u'username': u'space307', u'timestamp': 1513063742, u'like_id': 104399388}
{u'username': u'space307', u'timestamp': 1510818975, u'like_id': 101842997}
{u'username': u'space307', u'timestamp': 1510832134, u'like_id': 101870245}
{u'username': u'space307', u'timestamp': 1510140013, u'like_id': 101075270}
{u'username': u'space307', u'timestamp': 1510825707, u'like_id': 101857720}
{u'username': u'space307', u'timestamp': 1510147086, u'like_id': 101088377}
{u'username': u'space307', u'timestamp': 1512481574, u'like_id': 103728030}
{u'username'

{u'username': u'space307', u'timestamp': 1510917741, u'like_id': 101980429}
{u'username': u'space307', u'timestamp': 1511511058, u'like_id': 102624583}
{u'username': u'space307', u'timestamp': 1512475547, u'like_id': 103715773}
{u'username': u'space307', u'timestamp': 1510819021, u'like_id': 101843193}
{u'username': u'space307', u'timestamp': 1511284381, u'like_id': 102361231}
{u'username': u'space307', u'timestamp': 1512732880, u'like_id': 104082367}
{u'username': u'space307', u'timestamp': 1513095041, u'like_id': 104458216}
{u'username': u'space307', u'timestamp': 1512724693, u'like_id': 104069189}
{u'username': u'space307', u'timestamp': 1513006377, u'like_id': 104329338}
{u'username': u'space307', u'timestamp': 1512846873, u'like_id': 104183325}
{u'username': u'space307', u'timestamp': 1510687099, u'like_id': 101683714}


KeyboardInterrupt: 

### user dataset - likes_received_count

In [54]:
def printShotsLike(username):
    shots = db.shots.find({'username': username})

    num_likes = 0
    for s in shots[0]['shots']:
        if s['created_at'] <= 1512950340:
            num_likes = num_likes + s['likes_count']

    print 'shots likes: ', num_likes, '\n'

In [55]:
user = db.users.find({'created_at': {'$lt':1512950340}})
for i in range(100,200):
    print 'user: ', user[i]['username'], ' - like received: ', user[i]['likes_received_count']
    printShotsLike(user[i]['username'])

user:  visp  - like received:  3221
shots likes:  3222 

user:  chevail  - like received:  0
shots likes:  0 

user:  Sohieb  - like received:  0
shots likes:  0 

user:  minqi_luo  - like received:  296
shots likes:  296 

user:  owerrc  - like received:  0
shots likes:  0 

user:  YanboSolutions  - like received:  0
shots likes:  0 

user:  ricardogiesta  - like received:  0
shots likes:  0 

user:  santiagovaq  - like received:  307
shots likes:  306 

user:  JiarenHui  - like received:  172
shots likes:  173 

user:  liaoevan  - like received:  376
shots likes:  375 

user:  Viknayak  - like received:  0
shots likes:  0 

user:  XD  - like received:  642
shots likes:  642 

user:  OmarMamdouh25  - like received:  0
shots likes:  0 

user:  adamkporter  - like received:  1086
shots likes:  1087 

user:  lucschwab  - like received:  709
shots likes:  709 

user:  noelanijing  - like received:  1746
shots likes:  1747 

user:  shadsahm  - like received:  649
shots likes:  649 

user: 